(x_a) = (x_0, x_1, x_2, x_3) = (t, r, θ, ϕ)
g_ab=gco
[[exp(mu(r, t))        0          0           0      ]
 [0            -exp(lambda(r, t)) 0           0      ]
 [0                    0        -r**2         0      ]
 [0                    0          0 -r**2*sin(th)**2]]



In [46]:
import numpy as np
from sympy import *
import matplotlib as plt


def partial(x, d):  # derivada parcial em relação a indices
    if d == 0:
        return diff(x, t)
    elif d == 1:
        return diff(x, r)
    elif d == 2:
        return diff(x, th)
    elif d == 3:
        return diff(x, phi)


# variaveis
t, r, th, phi = symbols('t r theta phi')
lam = Function('lambda')(r, t)
mu = Function('mu')(r, t)
g1 = np.diag([exp(mu), -exp(lam), -r**2, -r**2*sin(th)**2])  # g_ab
g2 = np.diag([exp(-mu), -exp(-lam), -1/(r**2), -1/(r**2*sin(th)**2)])  # g^ab
ChristSymb = MutableDenseNDimArray(np.zeros((4, 4, 4)))
Curvature = MutableDenseNDimArray(np.zeros((4, 4, 4, 4)))  # R^a_bcd
LowCurvature = MutableDenseNDimArray(np.zeros((4, 4, 4, 4)))  # R_abcd
RicciTensor = MutableDenseNDimArray(np.zeros((4, 4)))  # R_ab
RicciScalar = 0  # R
EinsteinTensor = MutableDenseNDimArray(np.zeros((4, 4)))  # G_ab
upEinsteinTensor = MutableDenseNDimArray(np.zeros((4, 4)))  # G^a_b

for a in range(0, 4):
    for b in range(0, 4):
        for c in range(0, 4):
            for d in range(0, 4):
                ChristSymb[a, b, c] += 1/2*g2[a, d] * \
                    (partial(g1[d, c], b) +
                     partial(g1[d, b], c)-partial(g1[b, c], d))

for a in range(0, 4):
    for b in range(0, 4):
        for c in range(0, 4):
            for d in range(0, 4):
                termo1, termo2 = 0, 0
                for e in range(0, 4):
                    termo1 += ChristSymb[e, b, d]*ChristSymb[a, e, c]
                    termo2 += ChristSymb[e, b, c]*ChristSymb[a, e, d]
                Curvature[a, b, c, d] += partial(ChristSymb[a, b, d], c)-partial(
                    ChristSymb[a, b, c], d)+termo1-termo2


for a in range(0, 4):
    for b in range(0, 4):
        for c in range(0, 4):
            for d in range(0, 4):
                for e in range(0, 4):
                    LowCurvature[a, b, c, d] += g1[a, e]*Curvature[e, b, c, d]

for a in range(0, 4):
    for b in range(0, 4):
        for c in range(0, 4):
            RicciTensor[a, b] += Curvature[c, a, c, b]
            RicciTensor[a, b] = simplify(RicciTensor[a, b])
for a in range(0, 4):
    for b in range(0, 4):
        RicciScalar += g2[a, b]*RicciTensor[a, b]

for a in range(0, 4):
    for b in range(0, 4):
        EinsteinTensor[a, b] = RicciTensor[a, b]-1/2*g1[a, b]*RicciScalar

for a in range(0, 4):
    for b in range(0, 4):
        for c in range(0, 4):
            upEinsteinTensor[a, b] += g2[a, c]*EinsteinTensor[c, b]

file1 = open("ChristoffelSymbol.txt", "w+")
for a in range(0, 4):
    for b in range(0, 4):
        for c in range(b, 4):
            if ChristSymb[a, b, c] != 0:
                txt1 = latex(ChristSymb[a, b, c])
                txt1 = txt1.replace('0.5', '\\frac{1}{2}')
                txt1 = txt1.replace('1.0', '1')
                txt2 = "\n\\begin{equation}\n" + \
                    '\t\Gamma_{%d%d}^{%d}=' % (
                        a, b, c)+txt1+"\n\\end{equation}"
                file1.write(txt2)
file1.close()

file2 = open("LoweredCurvatureTensor.txt", "w+")
for a in range(0, 4):
    for b in range(0, 4):
        for c in range(0, 4):
            for d in range(0, 4):
                if LowCurvature[a, b, c, d] != 0:
                    txt1 = latex(simplify(LowCurvature[a, b, c, d]))
                    txt1 = txt1.replace('0.5', '\\frac{1}{2}')
                    txt1 = txt1.replace('1.0', '1')
                    txt1 = txt1.replace('0.25', '\\frac{1}{4}')
                    txt1 = txt1.replace(
                        '\lambda{\left(r,t \\right)}', '\lambda')
                    txt1 = txt1.replace('\mu{\left(r,t \\right)}', '\mu')
                    txt2 = "\n\\begin{equation}\n" + \
                        '\tR_{%d%d%d%d}=' % (a, b, c, d) + \
                        txt1+"\n\\end{equation}"
                    file2.write(txt2)
file2.close()


file3 = open("RicciTensor.txt", "w+")
for a in range(0, 4):
    for b in range(0, 4):
        if RicciTensor[a, b] != 0:
            txt1 = latex(simplify(RicciTensor[a, b]))
            txt1 = txt1.replace('0.5', '\\frac{1}{2}')
            txt1 = txt1.replace('1.0', '1')
            txt1 = txt1.replace('0.25', '\\frac{1}{4}')
            txt1 = txt1.replace('\lambda{\left(r,t \\right)}', '\lambda')
            txt1 = txt1.replace('\mu{\left(r,t \\right)}', '\mu')
            txt2 = "\n\\begin{equation}\n" + \
                '\tR_{%d%d}=' % (a, b)+txt1+"\n\\end{equation}"
            file3.write(txt2)
file3.close()

file4 = open("RicciScalar.txt", "w+")
txt1 = latex(simplify(RicciScalar))
txt1 = txt1.replace('0.5', '\\frac{1}{2}')
txt1 = txt1.replace('1.0', '1')
txt1 = txt1.replace('0.25', '\\frac{1}{4}')
txt1 = txt1.replace('\lambda{\left(r,t \\right)}', '\lambda')
txt1 = txt1.replace('\mu{\left(r,t \\right)}', '\mu')
txt2 = "\n\\begin{equation}\n" + \
    '\tR=' +txt1+"\n\\end{equation}"
file4.write(txt2)
file4.close()

file5 = open("EinsteinTensor.txt", "w+")
for a in range(0, 4):
    for b in range(0, 4):
        if EinsteinTensor[a,b] != 0:
            txt1 = latex(EinsteinTensor[a,b])
            txt1 = txt1.replace('0.5', '\\frac{1}{2}')
            txt1 = txt1.replace('1.0', '1')
            txt1 = txt1.replace('0.25', '\\frac{1}{4}')
            txt1 = txt1.replace('2.0', '2')
            txt1 = txt1.replace('\lambda{\left(r,t \\right)}', '\lambda')
            txt1 = txt1.replace('\mu{\left(r,t \\right)}', '\mu')
            txt2 = "\n\\begin{equation}\n" + \
                '\tG_{%d%d}=' % (a, b)+txt1+"\n\\end{equation}"
            file5.write(txt2)
file5.close()
file6 = open("upEinsteinTensor.txt", "w+")
for a in range(0, 4):
    for b in range(0, 4):
        if upEinsteinTensor[a,b] != 0:
            txt1 = latex(upEinsteinTensor[a,b])
            txt1 = txt1.replace('0.5', '\\frac{1}{2}')
            txt1 = txt1.replace('1.0', '1')
            txt1 = txt1.replace('0.25', '\\frac{1}{4}')
            txt1 = txt1.replace('2.0', '2')
            txt1 = txt1.replace('\lambda{\left(r,t \\right)}', '\lambda')
            txt1 = txt1.replace('\mu{\left(r,t \\right)}', '\mu')
            txt2 = "\n\\begin{equation}\n" + \
                '\tG^{%d}_{%d}=' % (a, b)+txt1+"\n\\end{equation}"
            file6.write(txt2)
file6.close()


